# Ноутбук с примером работы для грузов с реальными позициями (из 2gis)

In [ ]:
import sys
sys.path.append('../')

In [ ]:
import datetime
import random

import folium
import pandas as pd
from itertools import groupby
from scripts.cargo_delivery import delivery, PathBuildingResult
from scripts.data_model import Cargo, Tariff, TariffCost


Загрузка и валидация данных (грузы только между двумя точками)

In [ ]:
df_mos = pd.read_excel('../data/05_Грузы (только 2 точки) 2025-04-07.xlsx')
df_cpy = df_mos

In [ ]:
df_cpy['Кол-во заявок по ручному ID (со схожей датой сбора и городами доставки и отправления)'].unique()

In [ ]:
df_mos = df_cpy[df_cpy['Кол-во заявок по ручному ID (со схожей датой сбора и городами доставки и отправления)'] == 108]
df_mos = df_mos[df_mos['Количество точек'] == 2]

Вытаскиваем характеристики по массе объему по разным машинам

In [ ]:
cars_df = pd.read_excel('../data/Автомобили Логистика.xlsx', sheet_name='Лист2')
cars_df = cars_df[cars_df['Категория груза'] == 'Обычные']
car_name_to_mass_volume = {r['Наименование автомобиля']: (r['Грузоподъемность, (кг)'], r['Объем, (м³)']) for _, r in
                           cars_df.iterrows() if
                           'миксер' not in r['Наименование автомобиля'] and r['Грузоподъемность, (кг)'] <= 10000}
del cars_df

In [ ]:
car_name_to_mass_volume

Вытасуиваем тарифы

In [ ]:
cost_df = pd.read_excel('../data/Тарифы Логистика.xlsx')
cost_df = cost_df[cost_df.apply(lambda r: 'Москва' in r['Регион'] and 'и' not in r['Регион'], axis=1)]

In [ ]:
tariffs: list[Tariff] = []

for _, r in cost_df.iterrows():
    if r['Автомобиль'] not in car_name_to_mass_volume:
        continue
    tariffs.append(
        Tariff(
            id=r['Автомобиль'],
            mass=car_name_to_mass_volume[r['Автомобиль']][0],
            volume=car_name_to_mass_volume[r['Автомобиль']][1],
            cost_per_distance=[
                TariffCost(
                    min_dst_km=0,
                    max_dst_km=r['Минимальная поездка, км'],
                    cost_per_km=0,
                    fixed_cost=int(r['Стоимость мин. поездки, коп'] / 100)
                ),
                TariffCost(
                    min_dst_km=r['Минимальная поездка, км'],
                    max_dst_km=r['Максимальная протяженность маршрута, км'],
                    cost_per_km=int(r['Стоимость за км'] / 100),
                    fixed_cost=0
                )
            ])
    )

In [ ]:
del cost_df

In [ ]:
tariffs

Удаление дублей тарифов (опционально, просто в таблице много лишних дублей)

In [ ]:
def key(t: Tariff):
    return t.mass, t.volume


new_tariffs = []
for k, v in groupby(sorted(tariffs, key=key), key=key):
    v = list(v)[0]
    new_tariffs.append(v)
tariffs = new_tariffs

In [ ]:
len(tariffs)

# Загрузка графа OSM

In [ ]:
df_mos

In [ ]:
cargos: list[Cargo] = []


def get_time(param):
    param = str(param)
    if len(param) > 10:
        if '.' in param:
            return datetime.datetime.strptime(param, '%d.%m.%Y %H:%M:%S')
        else:
            return datetime.datetime.strptime(param, '%Y-%m-%d %H:%M:%S')
    else:
        return datetime.datetime.strptime(param, '%Y-%m-%d')

# парсим грузы из датафрейма
for i, r in df_mos.iterrows():
    
    start_node = r['Адрес отправления']
    end_node = r['Адрес назначения']
    
    mass = min(r['Вес, кг'], 100000)
    volume = min(r['Объем, м3'], 10)
    cm = r['ТРАНСПОРТ Грузоподъемность, (кг)']
    cv = r['ТРАНСПОРТ Объем, (м³)']
    d = Cargo(
        id=r['Номер заявки'],
        nodes=[start_node, end_node],
        mass=[mass, -mass],
        volume=[volume, -volume],
        service_time_minutes=[15, 15]
    )
    cargos.append(d)

In [ ]:
cargos[0]

In [ ]:
tariffs[0]

# Основной запуск модели

In [ ]:
result = delivery(
    cargos,
    tariffs
)

In [ ]:
result.simple_routes

In [ ]:
result.cargo_to_route

In [ ]:
len(cargos)

In [ ]:
for route in result.routes:
    print('________________________________________')
    print(f"car: {route.id} mass:{route.tariff.mass} volume: {route.tariff.volume}")
    max_mass = 1
    max_volume = 1
    print(f'mass: {max_mass}|{route.tariff.mass} == {max_mass / route.tariff.mass * 100:.2f}%')
    print(f'volume: {max_volume:.2f}|{route.tariff.volume:.2f} == {max_volume / route.tariff.volume * 100:.2f}%')

In [ ]:
def draw_on_map(data: PathBuildingResult,
                weight: float = 3.5,
                ) -> folium.Map:
    _g = data.cargo_graph
    u = list(_g.nodes())[0]
    u_x, u_y = _g.nodes()[u]['x'], _g.nodes()[u]['y']
    m: folium.Map = folium.Map(
        location=[u_x, u_y],
        zoom_start=11,
        tiles="cartodb positron"
    )  # Координаты города
    coords = {
    }

    for i, route in enumerate(data.routes):
        points_group = folium.FeatureGroup(name=f"points_{i}_{route.tariff.id}", show=False)
        points_group.add_to(m)
        path = route.path
        for i in range(len(path)):
            u = path[i]
            du = _g.nodes()[u]
            x, y = du['x'] + random.random() / 1000, du['y'] + random.random() / 1000
            coords[i] = (x, y)
            time = sum(data.cargo_graph.edges()[path[j], path[j + 1]]['time'] + data.cargo_graph.nodes()[path[j]]['service_time'] for j in range(i))
            
            test = f"""
            номер груза:       {path[i][0]}<br>
            номер ноды в графе:       {path[i][1]}<br>
            порядок посещения:      {i}<br>
            время:                  {time:.2f}<br>
            время обслуживание:     {data.cargo_graph.nodes()[path[i]]['service_time']:.2f}<br>
            """
            popup = folium.Popup(test, max_width=300, min_width=300)
            color = _g.nodes()[path[i]]['color'] if 'color' in _g.nodes()[path[i]] else 'red' if _g.nodes()[path[i]][
                                                                                                     'mass'] < 0 else 'blue'
            folium.CircleMarker(
                location=(x, y),
                radius=8,
                fill=True,
                fill_color=color,
                color=color,
                fill_opacity=0.7,
                popup=popup
            ).add_to(points_group)

        for i in range(len(path) - 1):
            u_x, u_y = coords[i]
            v_x, v_y = coords[i + 1]
            mass_by_edge = sum(_g.nodes()[path[j]]['mass'] for j in range(i + 1))
            volume_by_edge = sum(_g.nodes()[path[j]]['volume'] for j in range(i + 1))

            test = f"""
                масса на плече: {mass_by_edge:.2f}<br>
                объем на плече: {volume_by_edge:.2f}<br>
                длина плеча:    {_g.edges()[path[i], path[i + 1]]['length'] / 1000:.2f} <br>
                время:          {_g.edges()[path[i], path[i + 1]]['time']:.2f}
            """
            popup = folium.Popup(test, max_width=300, min_width=300)
            folium.PolyLine(
                [(u_x, u_y), (v_x, v_y)],
                weight=weight,
                color='blue',
                popup=popup
            ).add_to(points_group)
    folium.LayerControl().add_to(m)
    return m

In [ ]:
m = draw_on_map(result)

In [ ]:
m.show_in_browser()

In [ ]:
# m.save('map.html')